In [1]:
import numpy as np
import pandas as pd
import os
import tarfile
import io
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.preprocessing import OrdinalEncoder

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
i=0
for column in train:
  print("attribute ", i, "named ", column, "missing count: ", train[column].isna().sum())
  i += 1

attribute  0 named  Id missing count:  0
attribute  1 named  MSSubClass missing count:  0
attribute  2 named  MSZoning missing count:  0
attribute  3 named  LotFrontage missing count:  259
attribute  4 named  LotArea missing count:  0
attribute  5 named  Street missing count:  0
attribute  6 named  Alley missing count:  1369
attribute  7 named  LotShape missing count:  0
attribute  8 named  LandContour missing count:  0
attribute  9 named  Utilities missing count:  0
attribute  10 named  LotConfig missing count:  0
attribute  11 named  LandSlope missing count:  0
attribute  12 named  Neighborhood missing count:  0
attribute  13 named  Condition1 missing count:  0
attribute  14 named  Condition2 missing count:  0
attribute  15 named  BldgType missing count:  0
attribute  16 named  HouseStyle missing count:  0
attribute  17 named  OverallQual missing count:  0
attribute  18 named  OverallCond missing count:  0
attribute  19 named  YearBuilt missing count:  0
attribute  20 named  YearRemo

In [4]:
train = train.drop(columns=['LotFrontage','Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'HouseStyle'])
test = test.drop(columns=['LotFrontage','Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'HouseStyle'])

In [5]:
train = train.dropna(subset=['MasVnrType', 'Electrical', 'BsmtFinType2', 'BsmtExposure', 'GarageCond'])
i=0
for column in train:
  print("attribute ", i, "named ", column, "missing count: ", train[column].isna().sum())
  i += 1

attribute  0 named  Id missing count:  0
attribute  1 named  MSSubClass missing count:  0
attribute  2 named  MSZoning missing count:  0
attribute  3 named  LotArea missing count:  0
attribute  4 named  Street missing count:  0
attribute  5 named  LotShape missing count:  0
attribute  6 named  LandContour missing count:  0
attribute  7 named  Utilities missing count:  0
attribute  8 named  LotConfig missing count:  0
attribute  9 named  LandSlope missing count:  0
attribute  10 named  Neighborhood missing count:  0
attribute  11 named  Condition1 missing count:  0
attribute  12 named  Condition2 missing count:  0
attribute  13 named  BldgType missing count:  0
attribute  14 named  OverallQual missing count:  0
attribute  15 named  OverallCond missing count:  0
attribute  16 named  YearBuilt missing count:  0
attribute  17 named  YearRemodAdd missing count:  0
attribute  18 named  RoofStyle missing count:  0
attribute  19 named  RoofMatl missing count:  0
attribute  20 named  Exterior1s

In [6]:
print(len(train['Id']))

1338


In [7]:
train = train.drop(columns=['Id'])
test = test.drop(columns=['Id'])

In [8]:
train['MSSubClass'].head()

0    60
1    20
2    60
3    70
4    60
Name: MSSubClass, dtype: int64

In [9]:
type(train['LotShape'])

pandas.core.series.Series

In [10]:
train['LotShape'].replace(to_replace='IR3', value=0, inplace=True)

In [11]:
train['LotShape'].replace(to_replace='IR2', value=1, inplace=True)
train['LotShape'].replace(to_replace='IR1', value=2, inplace=True)
train['LotShape'].replace(to_replace='Reg', value=3, inplace=True)

In [12]:
test['LotShape'].replace(to_replace='IR3', value=0, inplace=True)
test['LotShape'].replace(to_replace='IR2', value=1, inplace=True)
test['LotShape'].replace(to_replace='IR1', value=2, inplace=True)
test['LotShape'].replace(to_replace='Reg', value=3, inplace=True)

In [13]:
train['LotShape'].head()

0    3
1    3
2    2
3    2
4    2
Name: LotShape, dtype: int64

In [14]:
train['Street'].replace(to_replace='Grvl', value=0, inplace=True)
train['Street'].replace(to_replace='Pave', value=1, inplace=True)

In [15]:
test['Street'].replace(to_replace='Grvl', value=0, inplace=True)
test['Street'].replace(to_replace='Pave', value=1, inplace=True)

In [16]:
train['LandContour'].replace(to_replace='Low', value=0, inplace=True)
train['LandContour'].replace(to_replace='HLS', value=1, inplace=True)
train['LandContour'].replace(to_replace='Bnk', value=2, inplace=True)
train['LandContour'].replace(to_replace='Lvl', value=3, inplace=True)

In [17]:
train['Utilities'].replace(to_replace='ELO', value=0, inplace=True)
train['Utilities'].replace(to_replace='NoSeWa', value=1, inplace=True)
train['Utilities'].replace(to_replace='NoSewr', value=2, inplace=True)
train['Utilities'].replace(to_replace='AllPub', value=3, inplace=True)


In [18]:
test['LandContour'].replace(to_replace='Low', value=0, inplace=True)
test['LandContour'].replace(to_replace='HLS', value=1, inplace=True)
test['LandContour'].replace(to_replace='Bnk', value=2, inplace=True)
test['LandContour'].replace(to_replace='Lvl', value=3, inplace=True)
test['Utilities'].replace(to_replace='ELO', value=0, inplace=True)
test['Utilities'].replace(to_replace='NoSeWa', value=1, inplace=True)
test['Utilities'].replace(to_replace='NoSewr', value=2, inplace=True)
test['Utilities'].replace(to_replace='AllPub', value=3, inplace=True)


In [19]:
train.drop(columns=['LotConfig'], inplace=True)

In [20]:
test.drop(columns=['LotConfig'], inplace=True)

In [21]:
train['LandSlope'].replace(to_replace='Sev', value=0, inplace=True)


In [22]:
train['LandSlope'].replace(to_replace='Mod', value=1, inplace=True)
train['LandSlope'].replace(to_replace='Gtl', value=2, inplace=True)

test['LandSlope'].replace(to_replace='Sev', value=0, inplace=True)
test['LandSlope'].replace(to_replace='Mod', value=1, inplace=True)
test['LandSlope'].replace(to_replace='Gtl', value=2, inplace=True)

In [23]:
train['ExterQual'].replace(to_replace='Po', value=0, inplace=True)
train['ExterQual'].replace(to_replace='Fa', value=1, inplace=True)
train['ExterQual'].replace(to_replace='TA', value=2, inplace=True)
train['ExterQual'].replace(to_replace='Gd', value=3, inplace=True)
train['ExterQual'].replace(to_replace='Ex', value=4, inplace=True)

test['ExterQual'].replace(to_replace='Po', value=0, inplace=True)
test['ExterQual'].replace(to_replace='Fa', value=1, inplace=True)
test['ExterQual'].replace(to_replace='TA', value=2, inplace=True)
test['ExterQual'].replace(to_replace='Gd', value=3, inplace=True)
test['ExterQual'].replace(to_replace='Ex', value=4, inplace=True)

In [24]:
train['ExterCond'].replace(to_replace='Po', value=0, inplace=True)
train['ExterCond'].replace(to_replace='Fa', value=1, inplace=True)
train['ExterCond'].replace(to_replace='TA', value=2, inplace=True)
train['ExterCond'].replace(to_replace='Gd', value=3, inplace=True)
train['ExterCond'].replace(to_replace='Ex', value=4, inplace=True)

test['ExterCond'].replace(to_replace='Po', value=0, inplace=True)
test['ExterCond'].replace(to_replace='Fa', value=1, inplace=True)
test['ExterCond'].replace(to_replace='TA', value=2, inplace=True)
test['ExterCond'].replace(to_replace='Gd', value=3, inplace=True)
test['ExterCond'].replace(to_replace='Ex', value=4, inplace=True)

In [25]:
train['BsmtQual'].replace(to_replace='NA', value=-1, inplace=True)
train['BsmtQual'].replace(to_replace='Po', value=0, inplace=True)
train['BsmtQual'].replace(to_replace='Fa', value=1, inplace=True)
train['BsmtQual'].replace(to_replace='TA', value=2, inplace=True)
train['BsmtQual'].replace(to_replace='Gd', value=3, inplace=True)
train['BsmtQual'].replace(to_replace='Ex', value=4, inplace=True)

test['BsmtQual'].replace(to_replace='NA', value=-1, inplace=True)
test['BsmtQual'].replace(to_replace='Po', value=0, inplace=True)
test['BsmtQual'].replace(to_replace='Fa', value=1, inplace=True)
test['BsmtQual'].replace(to_replace='TA', value=2, inplace=True)
test['BsmtQual'].replace(to_replace='Gd', value=3, inplace=True)
test['BsmtQual'].replace(to_replace='Ex', value=4, inplace=True)

In [26]:
train['BsmtCond'].replace(to_replace='NA', value=-1, inplace=True)
train['BsmtCond'].replace(to_replace='Po', value=0, inplace=True)
train['BsmtCond'].replace(to_replace='Fa', value=1, inplace=True)
train['BsmtCond'].replace(to_replace='TA', value=2, inplace=True)
train['BsmtCond'].replace(to_replace='Gd', value=3, inplace=True)
train['BsmtCond'].replace(to_replace='Ex', value=4, inplace=True)

test['BsmtCond'].replace(to_replace='NA', value=-1, inplace=True)
test['BsmtCond'].replace(to_replace='Po', value=0, inplace=True)
test['BsmtCond'].replace(to_replace='Fa', value=1, inplace=True)
test['BsmtCond'].replace(to_replace='TA', value=2, inplace=True)
test['BsmtCond'].replace(to_replace='Gd', value=3, inplace=True)
test['BsmtCond'].replace(to_replace='Ex', value=4, inplace=True)

In [27]:
train['BsmtExposure'].replace(to_replace='NA', value=0, inplace=True)
train['BsmtExposure'].replace(to_replace='No', value=1, inplace=True)
train['BsmtExposure'].replace(to_replace='Mn', value=2, inplace=True)
train['BsmtExposure'].replace(to_replace='Av', value=3, inplace=True)
train['BsmtExposure'].replace(to_replace='Gd', value=4, inplace=True)

test['BsmtExposure'].replace(to_replace='NA', value=0, inplace=True)
test['BsmtExposure'].replace(to_replace='No', value=1, inplace=True)
test['BsmtExposure'].replace(to_replace='Mn', value=2, inplace=True)
test['BsmtExposure'].replace(to_replace='Av', value=3, inplace=True)
test['BsmtExposure'].replace(to_replace='Gd', value=4, inplace=True)

In [28]:
train['BsmtFinType1'].replace(to_replace='NA', value=-1, inplace=True)
train['BsmtFinType1'].replace(to_replace='Unf', value=0, inplace=True)
train['BsmtFinType1'].replace(to_replace='Fa', value=1, inplace=True)
train['BsmtFinType1'].replace(to_replace='LwQ', value=2, inplace=True)
train['BsmtFinType1'].replace(to_replace='Rec', value=3, inplace=True)
train['BsmtFinType1'].replace(to_replace='BLQ', value=4, inplace=True)
train['BsmtFinType1'].replace(to_replace='ALQ', value=5, inplace=True)
train['BsmtFinType1'].replace(to_replace='GLQ', value=6, inplace=True)

test['BsmtFinType1'].replace(to_replace='NA', value=-1, inplace=True)
test['BsmtFinType1'].replace(to_replace='Unf', value=0, inplace=True)
test['BsmtFinType1'].replace(to_replace='Fa', value=1, inplace=True)
test['BsmtFinType1'].replace(to_replace='LwQ', value=2, inplace=True)
test['BsmtFinType1'].replace(to_replace='Rec', value=3, inplace=True)
test['BsmtFinType1'].replace(to_replace='BLQ', value=4, inplace=True)
test['BsmtFinType1'].replace(to_replace='ALQ', value=5, inplace=True)
test['BsmtFinType1'].replace(to_replace='GLQ', value=6, inplace=True)

In [29]:
train['BsmtFinType2'].replace(to_replace='NA', value=-1, inplace=True)
train['BsmtFinType2'].replace(to_replace='Unf', value=0, inplace=True)
train['BsmtFinType2'].replace(to_replace='Fa', value=1, inplace=True)
train['BsmtFinType2'].replace(to_replace='LwQ', value=2, inplace=True)
train['BsmtFinType2'].replace(to_replace='Rec', value=3, inplace=True)
train['BsmtFinType2'].replace(to_replace='BLQ', value=4, inplace=True)
train['BsmtFinType2'].replace(to_replace='ALQ', value=5, inplace=True)
train['BsmtFinType2'].replace(to_replace='GLQ', value=6, inplace=True)

test['BsmtFinType2'].replace(to_replace='NA', value=-1, inplace=True)
test['BsmtFinType2'].replace(to_replace='Unf', value=0, inplace=True)
test['BsmtFinType2'].replace(to_replace='Fa', value=1, inplace=True)
test['BsmtFinType2'].replace(to_replace='LwQ', value=2, inplace=True)
test['BsmtFinType2'].replace(to_replace='Rec', value=3, inplace=True)
test['BsmtFinType2'].replace(to_replace='BLQ', value=4, inplace=True)
test['BsmtFinType2'].replace(to_replace='ALQ', value=5, inplace=True)
test['BsmtFinType2'].replace(to_replace='GLQ', value=6, inplace=True)

In [30]:
train['HeatingQC'].replace(to_replace='Po', value=0, inplace=True)
train['HeatingQC'].replace(to_replace='Fa', value=1, inplace=True)
train['HeatingQC'].replace(to_replace='TA', value=2, inplace=True)
train['HeatingQC'].replace(to_replace='Gd', value=3, inplace=True)
train['HeatingQC'].replace(to_replace='Ex', value=4, inplace=True)

test['HeatingQC'].replace(to_replace='Po', value=0, inplace=True)
test['HeatingQC'].replace(to_replace='Fa', value=1, inplace=True)
test['HeatingQC'].replace(to_replace='TA', value=2, inplace=True)
test['HeatingQC'].replace(to_replace='Gd', value=3, inplace=True)
test['HeatingQC'].replace(to_replace='Ex', value=4, inplace=True)

In [31]:
train['CentralAir'].replace(to_replace='N', value=0, inplace=True)
train['CentralAir'].replace(to_replace='Y', value=1, inplace=True)

test['CentralAir'].replace(to_replace='N', value=0, inplace=True)
test['CentralAir'].replace(to_replace='Y', value=1, inplace=True)

In [32]:
train['KitchenQual'].replace(to_replace='Po', value=0, inplace=True)
train['KitchenQual'].replace(to_replace='Fa', value=1, inplace=True)
train['KitchenQual'].replace(to_replace='TA', value=2, inplace=True)
train['KitchenQual'].replace(to_replace='Gd', value=3, inplace=True)
train['KitchenQual'].replace(to_replace='Ex', value=4, inplace=True)

test['KitchenQual'].replace(to_replace='Po', value=0, inplace=True)
test['KitchenQual'].replace(to_replace='Fa', value=1, inplace=True)
test['KitchenQual'].replace(to_replace='TA', value=2, inplace=True)
test['KitchenQual'].replace(to_replace='Gd', value=3, inplace=True)
test['KitchenQual'].replace(to_replace='Ex', value=4, inplace=True)

In [33]:
train['Functional'].replace(to_replace='Sal', value=0, inplace=True)
train['Functional'].replace(to_replace='Sev', value=1, inplace=True)
train['Functional'].replace(to_replace='Maj2', value=2, inplace=True)
train['Functional'].replace(to_replace='Maj1', value=3, inplace=True)
train['Functional'].replace(to_replace='Mod', value=4, inplace=True)
train['Functional'].replace(to_replace='Min2', value=5, inplace=True)
train['Functional'].replace(to_replace='Min1', value=6, inplace=True)
train['Functional'].replace(to_replace='Typ', value=7, inplace=True)


test['Functional'].replace(to_replace='Sal', value=0, inplace=True)
test['Functional'].replace(to_replace='Sev', value=1, inplace=True)
test['Functional'].replace(to_replace='Maj2', value=2, inplace=True)
test['Functional'].replace(to_replace='Maj1', value=3, inplace=True)
test['Functional'].replace(to_replace='Mod', value=4, inplace=True)
test['Functional'].replace(to_replace='Min2', value=5, inplace=True)
test['Functional'].replace(to_replace='Min1', value=6, inplace=True)
test['Functional'].replace(to_replace='Typ', value=7, inplace=True)

In [34]:
train['GarageFinish'].replace(to_replace='NA', value=-1, inplace=True)
train['GarageFinish'].replace(to_replace='Unf', value=0, inplace=True)
train['GarageFinish'].replace(to_replace='RFn', value=1, inplace=True)
train['GarageFinish'].replace(to_replace='Fin', value=2, inplace=True)

test['GarageFinish'].replace(to_replace='NA', value=-1, inplace=True)
test['GarageFinish'].replace(to_replace='Unf', value=0, inplace=True)
test['GarageFinish'].replace(to_replace='RFn', value=1, inplace=True)
test['GarageFinish'].replace(to_replace='Fin', value=2, inplace=True)

In [35]:
train['GarageQual'].replace(to_replace='NA', value=-1, inplace=True)
train['GarageQual'].replace(to_replace='Po', value=0, inplace=True)
train['GarageQual'].replace(to_replace='Fa', value=1, inplace=True)
train['GarageQual'].replace(to_replace='TA', value=2, inplace=True)
train['GarageQual'].replace(to_replace='Gd', value=3, inplace=True)
train['GarageQual'].replace(to_replace='Ex', value=4, inplace=True)

test['GarageQual'].replace(to_replace='NA', value=-1, inplace=True)
test['GarageQual'].replace(to_replace='Po', value=0, inplace=True)
test['GarageQual'].replace(to_replace='Fa', value=1, inplace=True)
test['GarageQual'].replace(to_replace='TA', value=2, inplace=True)
test['GarageQual'].replace(to_replace='Gd', value=3, inplace=True)
test['GarageQual'].replace(to_replace='Ex', value=4, inplace=True)

In [36]:
train['GarageCond'].replace(to_replace='NA', value=-1, inplace=True)
train['GarageCond'].replace(to_replace='Po', value=0, inplace=True)
train['GarageCond'].replace(to_replace='Fa', value=1, inplace=True)
train['GarageCond'].replace(to_replace='TA', value=2, inplace=True)
train['GarageCond'].replace(to_replace='Gd', value=3, inplace=True)
train['GarageCond'].replace(to_replace='Ex', value=4, inplace=True)

test['GarageCond'].replace(to_replace='NA', value=-1, inplace=True)
test['GarageCond'].replace(to_replace='Po', value=0, inplace=True)
test['GarageCond'].replace(to_replace='Fa', value=1, inplace=True)
test['GarageCond'].replace(to_replace='TA', value=2, inplace=True)
test['GarageCond'].replace(to_replace='Gd', value=3, inplace=True)
test['GarageCond'].replace(to_replace='Ex', value=4, inplace=True)

In [37]:
train['PavedDrive'].replace(to_replace='N', value=0, inplace=True)
train['PavedDrive'].replace(to_replace='P', value=1, inplace=True)
train['PavedDrive'].replace(to_replace='Y', value=2, inplace=True)

test['PavedDrive'].replace(to_replace='N', value=0, inplace=True)
test['PavedDrive'].replace(to_replace='P', value=1, inplace=True)
test['PavedDrive'].replace(to_replace='Y', value=2, inplace=True)

Τωρα ξεκιναμε με τα One Hot:

'MSSubClass'
'MSZoning'
'Neighborhood'
'Condition1'
'Condition2'
'BldgType'
'RoofStyle'
'RoofMatl'
'Exterior1st'
'Exterior2nd'
'MasVnrType'
'Foundation'
'Heating'
'Electrical'
'GarageType'
'SaleType'
'SaleCondition'

In [38]:
MSSubClass_1hot = pd.get_dummies(train.MSSubClass)
MSZoning_1hot = pd.get_dummies(train.MSZoning)
Neighborhood_1hot = pd.get_dummies(train.Neighborhood)
Condition1_1hot = pd.get_dummies(train.Condition1)
Condition2_1hot = pd.get_dummies(train.Condition2)
BldgType_1hot = pd.get_dummies(train.BldgType)
RoofStyle_1hot = pd.get_dummies(train.RoofStyle)
RoofMatl_1hot = pd.get_dummies(train.RoofMatl)
Exterior1st_1hot = pd.get_dummies(train.Exterior1st)
Exterior2nd_1hot = pd.get_dummies(train.Exterior2nd)
MasVnrType_1hot = pd.get_dummies(train.MasVnrType)
Foundation_1hot = pd.get_dummies(train.Foundation)
Heating_1hot = pd.get_dummies(train.Heating)
Electrical_1hot = pd.get_dummies(train.Electrical)
GarageType_1hot = pd.get_dummies(train.GarageType)
SaleType_1hot = pd.get_dummies(train.SaleType)
SaleCondition_1hot = pd.get_dummies(train.SaleCondition)

In [39]:
test_MSSubClass_1hot = pd.get_dummies(test.MSSubClass)
test_MSZoning_1hot = pd.get_dummies(test.MSZoning)
test_Neighborhood_1hot = pd.get_dummies(test.Neighborhood)
test_Condition1_1hot = pd.get_dummies(test.Condition1)
test_Condition2_1hot = pd.get_dummies(test.Condition2)
test_BldgType_1hot = pd.get_dummies(test.BldgType)
test_RoofStyle_1hot = pd.get_dummies(test.RoofStyle)
test_RoofMatl_1hot = pd.get_dummies(test.RoofMatl)
test_Exterior1st_1hot = pd.get_dummies(test.Exterior1st)
test_Exterior2nd_1hot = pd.get_dummies(test.Exterior2nd)
test_MasVnrType_1hot = pd.get_dummies(test.MasVnrType)
test_Foundation_1hot = pd.get_dummies(test.Foundation)
test_Heating_1hot = pd.get_dummies(test.Heating)
test_Electrical_1hot = pd.get_dummies(test.Electrical)
test_GarageType_1hot = pd.get_dummies(test.GarageType)
test_SaleType_1hot = pd.get_dummies(test.SaleType)
test_SaleCondition_1hot = pd.get_dummies(test.SaleCondition)

In [40]:
train = pd.concat([train, MSSubClass_1hot], axis=1)
train = pd.concat([train, MSZoning_1hot], axis=1)
train = pd.concat([train, Neighborhood_1hot], axis=1)
train = pd.concat([train, Condition1_1hot], axis=1)
train = pd.concat([train, Condition2_1hot], axis=1)
train = pd.concat([train, BldgType_1hot], axis=1)
train = pd.concat([train, RoofStyle_1hot], axis=1)
train = pd.concat([train, RoofMatl_1hot], axis=1)
train = pd.concat([train, Exterior1st_1hot], axis=1)
train = pd.concat([train, Exterior2nd_1hot], axis=1)
train = pd.concat([train, MasVnrType_1hot], axis=1)
train = pd.concat([train, Foundation_1hot], axis=1)
train = pd.concat([train, Heating_1hot], axis=1)
train = pd.concat([train, Electrical_1hot], axis=1)
train = pd.concat([train, GarageType_1hot], axis=1)
train = pd.concat([train, SaleType_1hot], axis=1)
train = pd.concat([train, SaleCondition_1hot], axis=1)

In [41]:
test = pd.concat([test, test_MSSubClass_1hot], axis=1)
test = pd.concat([test, test_MSZoning_1hot], axis=1)
test = pd.concat([test, test_Neighborhood_1hot], axis=1)
test = pd.concat([test, test_Condition1_1hot], axis=1)
test = pd.concat([test, test_Condition2_1hot], axis=1)
test = pd.concat([test, test_BldgType_1hot], axis=1)
test = pd.concat([test, test_RoofStyle_1hot], axis=1)
test = pd.concat([test, test_RoofMatl_1hot], axis=1)
test = pd.concat([test, test_Exterior1st_1hot], axis=1)
test = pd.concat([test, test_Exterior2nd_1hot], axis=1)
test = pd.concat([test, test_MasVnrType_1hot], axis=1)
test = pd.concat([test, test_Foundation_1hot], axis=1)
test = pd.concat([test, test_Heating_1hot], axis=1)
test = pd.concat([test, test_Electrical_1hot], axis=1)
test = pd.concat([test, test_GarageType_1hot], axis=1)
test = pd.concat([test, test_SaleType_1hot], axis=1)
test = pd.concat([test, test_SaleCondition_1hot], axis=1)

In [42]:
train = train.drop(columns=['MSSubClass', 'MSZoning', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'SaleType', 'SaleCondition'])

In [43]:
test = test.drop(columns=['MSSubClass', 'MSZoning', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'SaleType', 'SaleCondition'])

In [44]:
train.describe()

,LotArea,Street,LotShape,LandContour,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,ConLw,New,Oth,WD,Abnorml,AdjLand,Alloca,Family,Normal,Partial
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,...,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,10706.294469,0.996263,2.574738,2.817638,2.998505,1.936472,6.219731,5.596413,1973.029148,1985.668909,...,0.002990,0.087444,0.000747,0.865471,0.064275,0.000747,0.005232,0.014948,0.825112,0.089686
std,10336.621126,0.061039,0.591024,0.603842,0.054677,0.278365,1.324472,1.078124,29.563540,20.296463,...,0.054615,0.282590,0.027338,0.341348,0.245334,0.027338,0.072168,0.121389,0.380013,0.285838
min,1300.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,2.000000,1880.000000,1950.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7744.000000,1.000000,2.000000,3.000000,3.000000,2.000000,5.000000,5.000000,1956.000000,1968.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,9600.000000,1.000000,3.000000,3.000000,3.000000,2.000000,6.000000,5.000000,1976.000000,1994.500000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,11760.750000,1.000000,3.000000,3.000000,3.000000,2.000000,7.000000,6.000000,2001.000000,2004.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,215245.000000,1.000000,3.000000,3.000000,3.000000,2.000000,10.000000,9.000000,2010.000000,2010.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [45]:
from sklearn.model_selection import train_test_split
X, y = train.drop(columns=['SalePrice']), train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [46]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()

lin_reg.fit(X_train_scaled, y_train)
lin_reg_predictions = lin_reg.predict(X_test_scaled)
lin_reg_mse = mean_squared_error(lin_reg_predictions, y_test)

In [48]:
print(lin_reg_mse)

3.553179340861086e+33


In [49]:
lin_reg_unscaled = LinearRegression()

lin_reg_unscaled.fit(X_train, y_train)
lin_reg_unscaled_predictions = lin_reg.predict(X_test)
lin_reg_unscaled_mse = mean_squared_error(lin_reg_unscaled_predictions, y_test)
print(lin_reg_unscaled_mse)

4.236981068426539e+38


## Πρεπει να κάνουμε mini batch... αλλά δεν υπάρχει έτοιμο, οπότε αύριο

In [50]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
X_train_scaled_reduced = pca.fit_transform(X_train_scaled)
X_test_scaled_reduced = pca.transform(X_test_scaled)

In [51]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor

poly_features = PolynomialFeatures(degree=2)
X_train_scaled_reduced_poly = poly_features.fit_transform(X_train_scaled_reduced)
X_test_scaled_reduced_poly = poly_features.fit_transform(X_test_scaled_reduced)

sgd_reg = SGDRegressor(max_iter=1000, tol=100, eta0=0.1)
sgd_reg.fit(X_train_scaled_reduced_poly, y_train)
sgd_poly_pred = sgd_reg.predict(X_test_scaled_reduced_poly)
sgd_poly_mse = mean_squared_error(sgd_poly_pred, y_test)
print(np.sqrt(sgd_poly_mse))

133837365280037.08
